In [22]:
import geopandas as gpd
import pandas as pd
import json
from pandas_ods_reader import read_ods

In [23]:
df = read_ods("data/CO2.ods", 1)
df.head()

,unnamed.1,unnamed.2,unnamed.3,unnamed.4,unnamed.5,unnamed.6,unnamed.7,unnamed.8,unnamed.9,unnamed.10,unnamed.11,unnamed.12
0,State,"Sum of 2,005","Sum of 2,006","Sum of 2,007","Sum of 2,008","Sum of 2,009","Sum of 2,010","Sum of 2,011","Sum of 2,012","Sum of 2,013","Sum of 2,014","Sum of 2,015"
1,None,2.5485e+08,2.6186e+08,2.67425e+08,2.67297e+08,2.64355e+08,2.66278e+08,2.70057e+08,2.91736e+08,2.92003e+08,2.40837e+08,2.43122e+08
2,Andaman and Nicobar Island,-7.10765e+06,-7.10741e+06,-7.10464e+06,-7.10861e+06,-7.11223e+06,-7.11409e+06,-7.12729e+06,-1.17063e+06,-1.17083e+06,357447,360883
3,Andhra Pradesh,5.34937e+06,5.84013e+06,6.13497e+06,6.42158e+06,5.46531e+06,6.5453e+06,6.22658e+06,-8.24616e+06,-9.63313e+06,-1.89579e+07,-1.97201e+07
4,Arunachal Pradesh,-2.06442e+07,-2.06566e+07,-2.06603e+07,-2.09495e+07,-2.09422e+07,-2.09163e+07,-2.07493e+07,5.46554e+06,5.48829e+06,6.42235e+06,6.27668e+06


In [24]:
def preprocess(df):
    cols = ['State']
    for d in df.columns:
        x = df[d][0]
    #     print(x)
        if(x!='State'):
            y = x.split(" ")
            yr = y[4].split('-')
            cols.append(y[3].replace(",",""))
    df.columns = cols
    state_yr = []
    state = ''
    for i in range(2,len(df)):
        for col in cols:
            if(col =='State'):
                state = df[col][i]
            else:
                year = col
                val = df[col][i]
    #             print(state, year,val)
                state_yr.append([state, year,val])
    df = pd.DataFrame(state_yr)
    df.head()
    df.columns = ['state','year','carbon_emission']
    return df


In [25]:
def delete_null(df):
    df = df.drop(df.index[0])
    df = df.dropna()
    df = df.reset_index(drop=True)
    return df

In [26]:
df = preprocess(df)

In [27]:
df = delete_null(df)


In [28]:
df

,state,year,carbon_emission
0,Andaman and Nicobar Island,2006,-7.107412e+06
1,Andaman and Nicobar Island,2007,-7.104641e+06
2,Andaman and Nicobar Island,2008,-7.108615e+06
3,Andaman and Nicobar Island,2009,-7.112226e+06
4,Andaman and Nicobar Island,2010,-7.114089e+06
...,...,...,...
390,West Bengal,2011,5.136389e+06
391,West Bengal,2012,2.431260e+07
392,West Bengal,2013,2.365977e+07
393,West Bengal,2014,2.339093e+07


In [29]:
from bokeh.io import output_notebook, show
from bokeh.palettes import inferno
output_notebook()

Loading BokehJS ...

In [ ]:
# Plot a complex chart with intearctive hover in a few lines of code

from bokeh.models import ColumnDataSource, HoverTool
from bokeh.plotting import figure
# from bokeh.sampledata.autompg import autompg_clean as df
from bokeh.transform import factor_cmap

df.state = df.state.astype(str)
df.year = df.year.astype(str)
# df.year = df.year.astype(str)
# print(df)
group = df.groupby(by=['state', 'year'])
group.head()
source = ColumnDataSource(group)

p = figure(plot_width=4000, plot_height=1000, title="State wise Carbon emissions",
           x_range=group, toolbar_location=None, tools="")

p.xgrid.grid_line_color = None
p.xaxis.axis_label = "state"
p.xaxis.major_label_orientation = 1.2

index_cmap = factor_cmap('state_year', palette=inferno(30), 
                         factors=sorted(df.year.unique()), end=1)

p.vbar(x='state_year', top='carbon_emission_mean', width=1, source=source,
       line_color="white", fill_color=index_cmap, 
       hover_line_color="darkgrey", hover_fill_color=index_cmap)

p.add_tools(HoverTool(tooltips=[("Carbon", "@carbon_emission_mean"), ("state, year", "@state_year")]))

show(p)

In [11]:
sta = 'Andaman and Nicobar Island'
df = df[df['state'].str.contains(sta)]


In [15]:
df.head()

,state,year,carbon_emission
0,Andaman and Nicobar Island,2006,-7.107412e+06
1,Andaman and Nicobar Island,2007,-7.104641e+06
2,Andaman and Nicobar Island,2008,-7.108615e+06
3,Andaman and Nicobar Island,2009,-7.112226e+06
4,Andaman and Nicobar Island,2010,-7.114089e+06


In [20]:
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.plotting import figure
# from bokeh.sampledata.autompg import autompg_clean as df
from bokeh.transform import factor_cmap

df.state = df.state.astype(str)
df.year = df.year.astype(str)
# df.year = df.year.astype(str)
# print(df)
group = df.groupby(by=['state', 'year'])
group.head()
source = ColumnDataSource(group)

p = figure(plot_width=300, plot_height=300, title="State wise Carbon emissions",
           x_range=group, toolbar_location=None, tools="")

p.xgrid.grid_line_color = None
p.xaxis.axis_label = "state"
p.xaxis.major_label_orientation = 1.2

index_cmap = factor_cmap('state_year', palette=['#2b83ba', '#abdda4', '#ffffbf', '#fdae61', '#d7191c','#FFA07A','#FA8072','#E9967A','#F08080','#CD5C5C'], 
                         factors=sorted(df.year.unique()), end=1)

p.vbar(x='state_year', top='carbon_emission_mean', width=1, source=source,
       line_color="white", fill_color='#CAB2D6', 
       hover_line_color="darkgrey", hover_fill_color=index_cmap)

p.add_tools(HoverTool(tooltips=[("Carbon", "@carbon_emission_mean"), ("state, year", "@state_year")]))

show(p)

/home/lenovo/.local/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [19]:
import pandas as pd

from bokeh.plotting import figure, output_file, show


df['year'] = pd.to_datetime(df['year'])

# output_file("datetime.html")

# create a new plot with a datetime axis type
p = figure(plot_width=600, plot_height=250, x_axis_type="datetime")

p.line(df['year'], df['carbon_emission'], color='navy', alpha=0.8)
p.circle(df['year'], df['carbon_emission'], fill_color="white", size=8)
# p.add_tools(HoverTool(tooltips=[("Carbon", "@ df['carbon_emission']"), ("state, year", "@ df['year']")]))


show(p)

/usr/lib/python3/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
